In [1]:
import sys
sys.path.insert(0, "../") # Add root/ to the path
# from src.magicmath.magicmath import MagicMath
from src.magicmath.data_generator import DataFrameGenerator
from pandas import concat
from pandas import DataFrame

In [2]:
# Generate Mock Data
data_generator = DataFrameGenerator()

columns = ["Market Cap", "EBIT", "Cash&Cash Equivalent", "Total Current Assets", "Property, Plant & Equipment", "Total Current Liabilities", "Total debt", "Depreciation"]
n_rows = 25
data = data_generator.generate_dataframe(columns, n_rows)
print(data.head())


   Market Cap      EBIT  Cash&Cash Equivalent  Total Current Assets  \
0    0.934250  0.245449              0.165287              0.082540   
1    0.344579  0.127187              0.750145              0.200297   
2    0.265696  0.667464              0.496675              0.810466   
3    0.053031  0.448494              0.070090              0.836723   
4    0.141074  0.391921              0.663037              0.198097   

   Property, Plant & Equipment  Total Current Liabilities  Total debt  \
0                     0.279983                   0.262300    0.757270   
1                     0.788127                   0.257291    0.460811   
2                     0.245572                   0.768953    0.888570   
3                     0.325028                   0.969809    0.591722   
4                     0.693499                   0.932290    0.936185   

   Depreciation  
0      0.126394  
1      0.341151  
2      0.698849  
3      0.614265  
4      0.596340  


In [3]:
# Calculate intermediate variables
net_fixed_assets = data["Property, Plant & Equipment"] - data["Depreciation"]
enterprise_value = data["Market Cap"] + data["Cash&Cash Equivalent"] - data["Total debt"]
net_working_capital = data["Total Current Assets"] - (data["Total Current Liabilities"] + data["Cash&Cash Equivalent"])
net_working_capital[net_working_capital < 0.0] = 0.0 # net_working_capital is non-negative 

In [4]:
# Calculate variables used in the ranking system of the magic formula
roic = data["EBIT"] / (net_fixed_assets + net_working_capital)
earnings_yield = data["EBIT"] / enterprise_value


In [5]:
# Calculate the rankings in order=ascending and then subtract them from the maximum rank plus one
rank_lambda = lambda series,ascending=True: (series.rank(ascending=ascending, method='first').max() + 1 - series.rank(ascending=ascending, method='first')).astype(int)
ranked_series = rank_lambda(roic)

# Higher is better for ROIC and Earnings yield
rankings_from_roic = rank_lambda(roic) 
rankings_from_earnings_yield = rank_lambda(earnings_yield)
# Magic formula, lower is better
scores_magicformula = rankings_from_roic + rankings_from_earnings_yield
rankings_magicformula = rank_lambda(scores_magicformula,False)

In [6]:
result = concat([roic, rankings_from_roic,
                 earnings_yield, rankings_from_earnings_yield,
                 scores_magicformula, rankings_magicformula],
                axis=1, join="inner",
                keys=["roic", "ranking_roic", "earnings_yield", "ranking_earnings_yield", "score_magicformula", "rankings_magicformula"])
result

,roic,ranking_roic,earnings_yield,ranking_earnings_yield,score_magicformula,rankings_magicformula
0,1.598092,5,0.717128,11,16,7
1,0.284549,11,0.200637,18,29,13
2,-1.472531,18,-5.288982,25,43,25
3,-1.550610,19,-0.957089,22,41,24
4,4.033809,1,-2.967434,24,25,11
5,0.146783,13,-0.547058,21,34,21
6,-3.450765,21,0.611715,15,36,22
7,0.274831,12,1.240315,7,19,9
8,0.957658,6,12.198881,1,7,1
9,1.912626,4,3.758039,4,8,2
